In [1]:
import os
import pickle
import numpy as np
import pandas as pd
from gsampler.h5io import load_from_h5
from IPython.display import display
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Deprecated: use ginkgo.Repo instead


##### Loading Kedan's dataFrames

In [14]:
LABEL = "neck_design_labels"
cv1 = pd.read_csv('../runFinetuneImagenet/models/lapel_design_labels/Xception90/validation_results.csv',
                 names=['fname', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])
display(cv1[:3])
tt1 = pd.read_csv('../runFinetuneImagenet/models/lapel_design_labels/Xception90/test_results_raw.csv',
                 names=['fname', 'label', 'x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8'])
display(tt1[:3])

,fname,x0,x1,x2,x3,x4,x5,x6,x7,x8
0,Images/lapel_design_labels/9592a06c983a46f87bf...,0.000015,0.001234,0.998145,0.000311,0.000295,NaN,NaN,NaN,NaN
1,Images/lapel_design_labels/da39c337916b962dc52...,0.004809,0.003855,0.989463,0.000289,0.001585,NaN,NaN,NaN,NaN
2,Images/lapel_design_labels/5be0cdac02b60e67af7...,0.000199,0.000032,0.999628,0.000117,0.000024,NaN,NaN,NaN,NaN


,fname,label,x0,x1,x2,x3,x4,x5,x6,x7,x8
0,Images/lapel_design_labels/518bcadaa97418d29c7...,lapel_design_labels,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,Images/lapel_design_labels/156228a4743bfd86cef...,lapel_design_labels,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,Images/lapel_design_labels/393f4962b5c2fbf42cb...,lapel_design_labels,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


##### Loading PKUGoodSpeed's dataFrames

In [17]:
cv2 = pickle.load(open('../pkugoodspeed/output/inception/'+LABEL+'_oof.pik', 'rb'))
cv3 = pd.read_pickle('../pkugoodspeed/output/kerasresnet/'+LABEL+'_oof.pik')
cv4 = pd.read_pickle('../pkugoodspeed/output/xception/'+LABEL+'_oof.pik')
display(cv2[:3])
display(cv3[:3])
display(cv4[:3])
print cv2.shape, np.array(cv2.pred.tolist()).shape
print cv3.shape, np.array(cv3.pred.tolist()).shape
print cv4.shape, np.array(cv4.pred.tolist()).shape

ImportError: No module named indexes.base

In [16]:
tt2 = pd.read_pickle('../pkugoodspeed/output/inception/'+LABEL+'_test.pik')
tt3 = pd.read_pickle('../pkugoodspeed/output/kerasresnet/'+LABEL+'_test.pik')
tt4 = pd.read_pickle('../pkugoodspeed/output/xception/'+LABEL+'_test.pik')
display(tt2[:3])
display(tt3[:3])
display(tt4[:3])

ImportError: No module named indexes.base

In [5]:
print cv1[['x0', 'x1', 'x2', 'x3', 'x4']].values.shape
print tt1[['x0', 'x1', 'x2', 'x3', 'x4']].values.shape

(420, 5)
(1591, 5)


##### Construct cv and test dataset

In [7]:
x_1 = cv1[['x0', 'x1', 'x2', 'x3', 'x4']].values
x_2 = np.array(cv2.pred.tolist())
x_3 = np.array(cv3.pred.tolist())
x_4 = np.array(cv4.pred.tolist())
assert x_1.shape == x_2.shape == x_3.shape == x_4.shape
cv_x = np.concatenate([x_1, x_2, x_3, x_4], axis = -1)
print cv_x.shape

(420, 20)


In [8]:
xt_1 = tt1[['x0', 'x1', 'x2', 'x3', 'x4']].values
xt_2 = np.array(tt2.pred.tolist())
xt_3 = np.array(tt3.pred.tolist())
xt_4 = np.array(tt4.pred.tolist())
assert xt_1.shape == xt_2.shape == xt_3.shape == xt_4.shape
tt_x = np.concatenate([xt_1, xt_2, xt_3, xt_4], axis = -1)
print tt_x.shape

(1591, 20)


In [9]:
cv_y = np.array(cv2.label.tolist())
cls_y = cv_y.argmax(axis=-1)
print cls_y[:5]

[3 3 3 0 1]


##### Construct a simple network

In [10]:
from keras.models import Model
from keras.layers import Dense, Input, Activation
from keras.optimizers import Adam

Using TensorFlow backend.


In [11]:
inlayer = Input((cv_x.shape[-1], ))
outlayer = Dense(cv_y.shape[-1], activation='softmax') (inlayer)
model = Model(inlayer, outlayer)
model.compile(optimizer=Adam(0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 105       
Total params: 105.0
Trainable params: 105.0
Non-trainable params: 0.0
_________________________________________________________________


In [42]:
res = model.fit(cv_x, cv_y, verbose=1, validation_split=0.5, batch_size=32,
                epochs=10)

Train on 210 samples, validate on 210 samples
Epoch 1/10
210/210 [==============================] - 0s - loss: 0.2697 - acc: 0.9095 - val_loss: 0.2970 - val_acc: 0.8857
Epoch 2/10
210/210 [==============================] - 0s - loss: 0.2695 - acc: 0.9095 - val_loss: 0.2970 - val_acc: 0.8857
Epoch 3/10
210/210 [==============================] - 0s - loss: 0.2694 - acc: 0.9095 - val_loss: 0.2969 - val_acc: 0.8857
Epoch 4/10
210/210 [==============================] - 0s - loss: 0.2693 - acc: 0.9095 - val_loss: 0.2969 - val_acc: 0.8857
Epoch 5/10
210/210 [==============================] - 0s - loss: 0.2692 - acc: 0.9095 - val_loss: 0.2968 - val_acc: 0.8857
Epoch 6/10
210/210 [==============================] - 0s - loss: 0.2691 - acc: 0.9095 - val_loss: 0.2968 - val_acc: 0.8857
Epoch 7/10
210/210 [==============================] - 0s - loss: 0.2690 - acc: 0.9095 - val_loss: 0.2968 - val_acc: 0.8857
Epoch 8/10
210/210 [==============================] - 0s - loss: 0.2689 - acc: 0.9095 - val_l

##### Grid Search

In [43]:
def getAccu(c1, c2, c3):
    if c1 + c2 + c3 > 1:
        return 0.
    c4 = 1. - c1 - c2 - c3
    x_comb = c1*x_1 + c2*x_2 + c3*x_3 + c4*x_4
    cls_x = x_comb.argmax(axis=-1)
    return 1.*(cls_x == cls_y).sum()/len(cls_x)
print(getAccu(0.6, 0.2, 0.1))

0.878571428571


In [44]:
%%time
c10, c20, c30 = 1., 0., 0.
values = np.arange(0., 1., 0.01)
acc = getAccu(c10, c20, c30)
print acc
for c1 in values:
    for c2 in values:
        for c3 in values:
            tmp = getAccu(c1, c2, c3)
            if tmp > acc:
                c10 = c1
                c20 = c2
                c30 = c3
                acc = tmp
print c10, c20, c30, 1.-c10-c20-c30, acc

0.871428571429
0.41 0.18 0.18 0.23 0.919047619048
CPU times: user 10.9 s, sys: 2.84 ms, total: 10.9 s
Wall time: 10.9 s


##### Construct the submission dataframe

In [45]:
pred_1 = model.predict(tt_x)
pred_2 = xt_1
pred_3 = c10*xt_1 + c20*xt_2 + c30*xt_3 + (1.-c10-c20-c30)*xt_4
print pred_1.shape, pred_2.shape, pred_3.shape
print pred_1[:4]
print pred_2[:4]
print pred_3[:4]

(1591, 5) (1591, 5) (1591, 5)
[[ 0.15724981  0.02569449  0.1103683   0.03293017  0.67375726]
 [ 0.03209651  0.0055634   0.49391326  0.46567631  0.0027505 ]
 [ 0.02782341  0.75695711  0.18559535  0.00762052  0.02200356]
 [ 0.13707365  0.01845366  0.53188407  0.30077338  0.01181515]]
[[ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]]
[[  1.37517617e-01   1.13642802e-02   4.18202390e-01   2.05279304e-02
    4.12387818e-01]
 [  8.27991516e-03   5.72268287e-03   4.50218145e-01   5.34199461e-01
    1.57981777e-03]
 [  6.57960931e-04   5.85352466e-01   4.12529398e-01   9.83920840e-04
    4.76280420e-04]
 [  5.84519858e-02   4.59494842e-02   4.59479582e-01   3.09512805e-01
    1.26606192e-01]]


In [46]:
assert (np.array(tt1.fname.tolist()) != np.array(tt2.fname.tolist())).sum() == 0

In [47]:
def _tostring(x):
    return ';'.join(["{0:.4f}".format(round(v, 4)) for v in x])

In [48]:
df1 = pd.DataFrame({
    'fname': tt2.fname.tolist(),
    'label': [LABEL for i in range(len(tt2))],
    'pred': [_tostring(x) for x in pred_1]
})
df1 = df1[['fname', 'label', 'pred']]
display(df1[:3])

,fname,label,pred
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels,0.1572;0.0257;0.1104;0.0329;0.6738
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,0.0321;0.0056;0.4939;0.4657;0.0028
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels,0.0278;0.7570;0.1856;0.0076;0.0220


In [49]:
df2 = pd.DataFrame({
    'fname': tt2.fname.tolist(),
    'label': [LABEL for i in range(len(tt2))],
    'pred': [_tostring(x) for x in pred_2]
})
df2 = df2[['fname', 'label', 'pred']]
display(df2[:3])

,fname,label,pred
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels,0.0000;0.0000;1.0000;0.0000;0.0000
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,0.0000;0.0000;1.0000;0.0000;0.0000
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels,0.0000;0.0000;1.0000;0.0000;0.0000


In [52]:
df3 = pd.DataFrame({
    'fname': tt2.fname.tolist(),
    'label': [LABEL for i in range(len(tt2))],
    'pred': [_tostring(x) for x in pred_3]
})
df3 = df3[['fname', 'label', 'pred']]
display(df3[:3])

,fname,label,pred
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels,0.1375;0.0114;0.4182;0.0205;0.4124
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels,0.0083;0.0057;0.4502;0.5342;0.0016
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels,0.0007;0.5854;0.4125;0.0010;0.0005


In [53]:
path = '../pkugoodspeed/data/merge/'
df1.to_csv(path+LABEL+'_v1.csv', index=False)
df2.to_csv(path+LABEL+'_v2.csv', index=False)
df3.to_csv(path+LABEL+'_v3.csv', index=False)